# General settings

In [1]:
import sys
import math
from scipy.optimize import minimize, bisect
import numpy as np
from numpy import linalg as nplin

In [2]:
from ekdist import eklib

In [3]:
# LOAD DATA.
filename = "./tests/intervals.txt"
intervals = np.loadtxt(filename)

## Fit multi-exponential pdf

In [4]:
theta = [0.036, 1.1, 0.20]
print('Start LogLikelihood =', eklib.LLexp(theta, np.array(intervals)))

Start LogLikelihood = 87.31806715582867


In [5]:
res = minimize(eklib.LLexp, theta, args=np.array(intervals), method='Nelder-Mead')
print (res)

 final_simplex: (array([[0.03700718, 1.07302608, 0.19874548],
       [0.03701042, 1.07300003, 0.19879291],
       [0.03701506, 1.07312038, 0.19880849],
       [0.03700726, 1.07309781, 0.19883809]]), array([87.28828773, 87.28828814, 87.28828846, 87.28828878]))
           fun: 87.28828773327726
       message: 'Optimization terminated successfully.'
          nfev: 88
           nit: 47
        status: 0
       success: True
             x: array([0.03700718, 1.07302608, 0.19874548])


### Calculate approximate SD

In [6]:
hess = eklib.hessian(res.x, eklib.LLexp, np.array(intervals))
print(hess)

[[8475.49606292  -92.21425006 -626.06648917]
 [ -92.21425006   71.5478832   -36.17166531]
 [-626.06648917  -36.17166531  501.13966047]]


In [7]:
cov = nplin.inv(hess)
print(cov)

[[0.00013478 0.00026864 0.00018777]
 [0.00026864 0.01504143 0.00142128]
 [0.00018777 0.00142128 0.00233261]]


In [8]:
appSD = np.sqrt(cov.diagonal())
print(appSD)

[0.01160948 0.1226435  0.04829715]


In [9]:
eklib.correlation_matrix(cov)

array([[1.        , 0.18867387, 0.33487997],
       [0.18867387, 1.        , 0.23994593],
       [0.33487997, 0.23994593, 1.        ]])

In [10]:
sd = eklib.approximate_SD(res.x, eklib.LLexp, np.array(intervals))

Tau = 0.037007; approximate SD = 0.011609
Area= 0.198745; approximate SD = 0.048297
Tau = 1.073026; approximate SD = 0.122644
Area= 0.801255; approximate SD = 0.048297
